In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import keras
from keras.preprocessing.image import ImageDataGenerator,load_img,img_to_array
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,UpSampling2D,Flatten,Dropout,Dense,BatchNormalization,Reshape

In [2]:
def diff(a,b):
    c=np.zeros(a.shape);
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                c[i][j][k]=(1+a[i][j][k]-b[i][j][k])/2;
    return(c);

In [3]:
def reform(a,c):
    b=np.zeros(a.shape);
    for i in range(a.shape[0]):
        for j in range(a.shape[1]):
            for k in range(a.shape[2]):
                b[i][j][k]=(1+a[i][j][k])-2*c[i][j][k];
    return(b);

In [ ]:
import os

x_train=[]
y_train=[]
x_test=[]
y_test=[]

path_train_low='../dataSets/ref/our485/low/'
path_train_high='../dataSets/ref/our485/high/'
path_test_low='../dataSets/ref/eval15/low/'
path_test_high='../dataSets/ref/eval15/high/'

print('\nLoding Train dataset :')
i=0;
for img in os.listdir(path_train_low):
    img_low_path=os.path.join(path_train_low, img);
    img_high_path=os.path.join(path_train_high, img);
    
    img_low=load_img(img_low_path, target_size=(64,64));
    img_low=img_to_array(img_low)/255.;
    
    img_high=load_img(img_high_path, target_size=(64,64));
    img_high=img_to_array(img_high)/255.;
        
    Rl,Gl,Bl=cv2.split(img_low);
    Rh,Gh,Bh=cv2.split(img_high);

    Rl=Rl.reshape((64,64,1));
    Rh=Rh.reshape((64,64,1));
    Rd=diff(Rl,Rh);
    
    x_train.append(Rl);
    y_train.append(Rd);
    
    i=i+1;
    print("{:.2f}".format(i/4.89)+' %',end='\r');

print('\nLoding Test dataset :')
i=0;
for img in os.listdir(path_test_low):
    img_low_path=os.path.join(path_test_low, img);
    img_high_path=os.path.join(path_test_high, img);
    
    img_low=load_img(img_low_path, target_size=(64,64));
    img_low=img_to_array(img_low)/255.;
    
    img_high=load_img(img_high_path, target_size=(64,64));
    img_high=img_to_array(img_high)/255.;
        
    Rl,Gl,Bl=cv2.split(img_low);
    Rh,Gh,Bh=cv2.split(img_high);
    
    Rl=Rl.reshape((64,64,1));
    Rh=Rh.reshape((64,64,1));
    Rd=diff(Rl,Rh);
    
    x_test.append(Rl);
    y_test.append(Rd);
    
    i=i+1;
    print("{:.2f}".format(i/0.14)+' %',end='\r');

x_train=np.array(x_train, dtype="float32");
y_train=np.array(y_train, dtype="float32");

x_test=np.array(x_test, dtype="float32");
y_test=np.array(y_test, dtype="float32");


Loding Train dataset :


In [ ]:
x_train.shape

In [ ]:
fig=plt.figure()

f=fig.add_subplot(1,2,1)
s=fig.add_subplot(1,2,2)

f.imshow(x_test[2])
s.imshow(y_test[2])

In [ ]:
# RED
model=Sequential()

model.add(Conv2D(128,(3,3),padding='same',activation='relu',input_shape=(64,64,1)))
model.add(UpSampling2D((2,2)))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(Conv2D(64,(3,3),padding='same',activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Conv2D(3,(1,1),padding='same',activation='sigmoid'))

model.summary()

In [ ]:
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau

checkpoint=ModelCheckpoint('models/ref/ref_cnn1_red.h5',
                            monitor='val_loss',
                            mode='min',
                            save_best_only=True,
                            verbose=1
                          )
earlystop=EarlyStopping(monitor='val_loss',
                        patience=10,
                        min_delta=0,
                        restore_best_weights=True,
                        verbose=1
                       )
reduce_lr=ReduceLROnPlateau(monitor='val_loss',
                            factor=0.2,
                            patience=10,
                            min_delta=0.0001,
                            verbose=1
                           )
callbacks=[checkpoint,earlystop,reduce_lr]

model.compile(loss=keras.losses.MeanSquaredError(),
              optimizer='adam',
              metrics=['accuracy']
             )

history=model.fit(
    x=x_train,
    y=y_train,
    batch_size=32,
    epochs=200,
    validation_data=(x_test, y_test),
    callbacks=callbacks
    )

In [ ]:
# train_datagen=ImageDataGenerator(
#         rescale=1./255,
#         rotation_range=30,
#         width_shift_range=0.2,
#         height_shift_range=0.2,
#         shear_range=0.2,
#         zoom_range=0.2,
#         horizontal_flip=True,
#         fill_mode='nearest'
#         )
# history=model.fit(
#     train_datagen.flow(x_train,y_train),
#     batch_size=32,
#     epochs=200,
#     validation_data=(x_test, y_test),
#     callbacks=callbacks
#     )

In [ ]:
plt.grid()

plt.plot(history.history['loss'],'--^r',label="loss")
plt.plot(history.history['accuracy'],'--^g',label='accuracy')
plt.plot(history.history['val_loss'],'-or',label='val_loss')
plt.plot(history.history['val_accuracy'],'-og',label='val_accuracy')
plt.legend()

In [ ]:
fig=plt.figure()

def check_result(n):
    fig.add_subplot(1,2,1).imshow(y_test[n])
    fig.add_subplot(1,2,2).imshow(reform(x_test[n],model.predict(x_test)[n]))

check_result(2)

In [ ]:
y_test[2]